In [118]:
from pathlib import Path
import numpy as np
import torch
from typing import List
from torch.nn.utils.rnn import pad_sequence
from mltrainer import rnn_models, Trainer
from torch import optim

from mads_datasets import datatools
import mltrainer
mltrainer.__version__

'0.2.5'

# 1 Iterators
We will be using an interesting dataset. [link](https://tev.fbk.eu/resources/smartwatch)

From the site:
> The SmartWatch Gestures Dataset has been collected to evaluate several gesture recognition algorithms for interacting with mobile applications using arm gestures. Eight different users performed twenty repetitions of twenty different gestures, for a total of 3200 sequences. Each sequence contains acceleration data from the 3-axis accelerometer of a first generation Sony SmartWatch™, as well as timestamps from the different clock sources available on an Android device. The smartwatch was worn on the user's right wrist. 


In [119]:
from mads_datasets import DatasetFactoryProvider, DatasetType
from mltrainer.preprocessors import PaddedPreprocessor
preprocessor = PaddedPreprocessor()

gesturesdatasetfactory = DatasetFactoryProvider.create_factory(DatasetType.GESTURES)
streamers = gesturesdatasetfactory.create_datastreamer(batchsize=32, preprocessor=preprocessor)
train = streamers["train"]
valid = streamers["valid"]

2025-09-27 14:07:28.865 | INFO     | mads_datasets.base:download_data:121 - Folder already exists at /Users/wouter/.cache/mads_datasets/gestures
100%|██████████| 651/651 [00:00<00:00, 12963.21it/s]


In [120]:
len(train), len(valid)

(81, 20)

In [121]:
trainstreamer = train.stream()
validstreamer = valid.stream()
x, y = next(iter(trainstreamer))
x.shape, y

(torch.Size([32, 32, 3]),
 tensor([ 6,  6, 17,  6, 13,  1, 12,  3,  0,  7,  2, 11, 18, 16, 17,  0,  9,  4,
          7,  4, 13,  7, 15,  7, 13, 19,  5,  8, 17,  7,  0, 16]))

Can you make sense of the shape?
What does it mean that the shapes are sometimes (32, 27, 3), but a second time might look like (32, 30, 3)? In other words, the second (or first, if you insist on starting at 0) dimension changes. Why is that? How does the model handle this? Do you think this is already padded, or still has to be padded?


# 2 Excercises
Lets test a basemodel, and try to improve upon that.

Fill the gestures.gin file with relevant settings for `input_size`, `hidden_size`, `num_layers` and `horizon` (which, in our case, will be the number of classes...)

As a rule of thumbs: start lower than you expect to need!

In [122]:
from mltrainer import TrainerSettings, ReportTypes
from mltrainer.metrics import Accuracy

accuracy = Accuracy()


In [ ]:
import inspect
from mltrainer import rnn_models

base_kwargs = dict(
    input_size=3,
    hidden_size=256,    
    num_layers=2,       
    horizon=20,
    bidirectional=True,  
    dropout=0.3          
)

sig = inspect.signature(rnn_models.BaseRNN)
base_kwargs = {k: v for k, v in base_kwargs.items() if k in sig.parameters}

model = rnn_models.BaseRNN(**base_kwargs)
model

BaseRNN(
  (rnn): RNN(3, 256, num_layers=2, batch_first=True)
  (linear): Linear(in_features=256, out_features=20, bias=True)
)

Test the model. What is the output shape you need? Remember, we are doing classification!

In [124]:
yhat = model(x)
yhat.shape

torch.Size([32, 20])

Test the accuracy

In [125]:
accuracy(y, yhat)

0.125

What do you think of the accuracy? What would you expect from blind guessing?

Check shape of `y` and `yhat`

In [126]:
yhat.shape, y.shape

(torch.Size([32, 20]), torch.Size([32]))

And look at the output of yhat

In [127]:
yhat[0]

tensor([ 0.1585,  0.1464,  0.0755, -0.0136, -0.1549, -0.0172,  0.0251, -0.0214,
        -0.1901, -0.0903, -0.0899, -0.0935,  0.0008, -0.0962,  0.0487,  0.0693,
         0.1096,  0.0813,  0.0261, -0.0608], grad_fn=<SelectBackward0>)

Does this make sense to you? If you are unclear, go back to the classification problem with the MNIST, where we had 10 classes.

We have a classification problem, so we need Cross Entropy Loss.
Remember, [this has a softmax built in](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html) 

In [128]:
loss_fn = torch.nn.CrossEntropyLoss()
loss = loss_fn(yhat, y)
loss

tensor(2.9804, grad_fn=<NllLossBackward0>)

In [129]:
import torch
if torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = torch.device("mps")
    print("Using MPS")
elif torch.cuda.is_available():
    device = "cuda:0"
    print("using cuda")
else:
    device = "cpu"
    print("using cpu")

# on my mac, at least for the BaseRNN model, mps does not speed up training
# probably because the overhead of copying the data to the GPU is too high
# so i override the device to cpu
device = "cpu"
# however, it might speed up training for larger models, with more parameters

Using MPS


Set up the settings for the trainer and the different types of logging you want

In [130]:
settings = TrainerSettings(
    epochs=35, # increase this to about 100 for training
    metrics=[accuracy],
    logdir=Path("gestures"),
    train_steps=len(train),
    valid_steps=len(valid),
    reporttypes=[ReportTypes.TOML, ReportTypes.TENSORBOARD, ReportTypes.MLFLOW],
    scheduler_kwargs={"factor": 0.5, "patience": 5},
    earlystop_kwargs = {
        "save": False, # save every best model, and restore the best one
        "verbose": True,
        "patience": 8, # number of epochs with no improvement after which training will be stopped
        "delta": 0.0, # minimum change to be considered an improvement
    }
)
settings

epochs: 35
metrics: [Accuracy]
logdir: gestures
train_steps: 81
valid_steps: 20
reporttypes: [<ReportTypes.TOML: 'TOML'>, <ReportTypes.TENSORBOARD: 'TENSORBOARD'>, <ReportTypes.MLFLOW: 'MLFLOW'>]
optimizer_kwargs: {'lr': 0.001, 'weight_decay': 1e-05}
scheduler_kwargs: {'factor': 0.5, 'patience': 5}
earlystop_kwargs: {'save': False, 'verbose': True, 'patience': 8, 'delta': 0.0}

In [131]:
import torch.nn as nn
import torch
from torch import Tensor
from dataclasses import dataclass

@dataclass
class ModelConfig:
    input_size: int
    hidden_size: int
    num_layers: int
    output_size: int
    dropout: float = 0.0

class GRUmodel(nn.Module):
    def __init__(
        self,
        config,
    ) -> None:
        super().__init__()
        self.config = config
        self.rnn = nn.GRU(
            input_size=config.input_size,
            hidden_size=config.hidden_size,
            dropout=config.dropout,
            batch_first=True,
            num_layers=config.num_layers,
        )
        self.linear = nn.Linear(config.hidden_size, config.output_size)

    def forward(self, x: Tensor) -> Tensor:
        x, _ = self.rnn(x)
        last_step = x[:, -1, :]
        yhat = self.linear(last_step)
        return yhat

In [132]:
config = ModelConfig(
    input_size=3,
    hidden_size=64,
    num_layers=1,
    output_size=20,
    dropout=0.0,
)


In [134]:
import mlflow
from datetime import datetime

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("gestures")
modeldir = Path("gestures").resolve()
if not modeldir.exists():
    modeldir.mkdir(parents=True)

with mlflow.start_run():
    mlflow.set_tag("model", "modelname-here")
    mlflow.set_tag("dev", "your-name-here")
    config = ModelConfig(
        input_size=3,
        hidden_size=256,
        num_layers=2,
        output_size=20,
        dropout=0.1,
    )

    model = GRUmodel(
        config=config,
    )

    trainer = Trainer(
        model=model,
        settings=settings,
        loss_fn=loss_fn,
        optimizer=optim.Adam,
        traindataloader=trainstreamer,
        validdataloader=validstreamer,
        scheduler=optim.lr_scheduler.ReduceLROnPlateau,
        device=device,
    )
    trainer.loop()

    if not settings.earlystop_kwargs["save"]:
        tag = datetime.now().strftime("%Y%m%d-%H%M-")
        modelpath = modeldir / (tag + "model.pt")
        torch.save(model, modelpath)

2025-09-27 14:08:24.289 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to gestures/20250927-140824
2025-09-27 14:08:24.290 | INFO     | mltrainer.trainer:__init__:68 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
100%|██████████| 81/81 [00:01<00:00, 44.62it/s]
2025-09-27 14:08:26.299 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 2.3502 test 1.9266 metric ['0.3219']
100%|██████████| 81/81 [00:01<00:00, 45.58it/s]
2025-09-27 14:08:28.261 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 1.5126 test 1.1881 metric ['0.5172']
100%|██████████| 81/81 [00:01<00:00, 47.18it/s]
2025-09-27 14:08:30.168 | INFO     | mltrainer.trainer:report:209 - Epoch 2 train 0.9210 test 0.7017 metric ['0.7641']
100%|██████████| 81/81 [00:01<00:00, 46.15it/s]
2025-09-27 14:08:32.111 | INFO     | mltrainer.trainer:report:209 - Epoch 3 train 0.4312 test 0.3703 metric ['0.9203']
100%|██████████| 81/81 [00:01<00:00, 46.27it/s]
2025-09-27 14:08:34.03

Try to update the code above by changing the hyperparameters.
    
To discern between the changes, also modify the tag mlflow.set_tag("model", "new-tag-here") where you add
a new tag of your choice. This way you can keep the models apart.

In [135]:
trainer.loop() # if you want to pick up training, loop will continue from the last epoch

100%|██████████| 81/81 [00:01<00:00, 45.42it/s]
2025-09-27 14:09:14.610 | INFO     | mltrainer.trainer:report:175 - Resuming epochs from previous training at 49
2025-09-27 14:09:14.629 | INFO     | mltrainer.trainer:report:209 - Epoch 49 train 0.0054 test 0.0113 metric ['0.9969']
2025-09-27 14:09:14.629 | INFO     | mltrainer.trainer:loop:103 - Interrupting loop due to early stopping patience.
2025-09-27 14:09:14.630 | INFO     | mltrainer.trainer:loop:108 - early_stopping_save was false, using latest model.Set to true to retrieve best model.
  0%|          | 0/35 [00:01<?, ?it/s]


In [136]:
mlflow.end_run()